In [46]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [47]:
data_row = pd.read_csv('C:/Users/82108/Desktop/jw/프로젝트/상관분석/(정규화전)학습데이터.csv')

In [48]:
data = data_row.copy()
data.head(5)

,Unnamed: 0,Gu,Dong,Population,Pet_Owners,Total_Registrations,hospital,product,beauty,h+p,h+b,p+b,h+p+b
0,0,동대문구,이문동,41423,1344.0,2373.0,2,1,1,3,3,2,4
1,1,동대문구,장안동,71052,2745.0,3584.0,8,9,1,17,9,10,18
2,2,동대문구,전농동,51329,1559.0,1960.0,3,1,2,4,5,3,6
3,3,동대문구,회기동,13653,285.0,335.0,1,1,1,2,2,2,3
4,4,동대문구,답십리동,56138,1873.0,2538.0,6,2,1,8,7,3,9


In [49]:
# 필요없는 열 제거
data = data.drop(['Unnamed: 0','Gu','Dong','product','beauty','p+b'],axis=1)
data.head(1)

,Population,Pet_Owners,Total_Registrations,hospital,h+p,h+b,h+p+b
0,41423,1344.0,2373.0,2,3,3,4


In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Population           87 non-null     int64  
 1   Pet_Owners           87 non-null     float64
 2   Total_Registrations  87 non-null     float64
 3   hospital             87 non-null     int64  
 4   h+p                  87 non-null     int64  
 5   h+b                  87 non-null     int64  
 6   h+p+b                87 non-null     int64  
dtypes: float64(2), int64(5)
memory usage: 4.9 KB


In [51]:

#data['Pet_Owners'] = data['Pet_Owners'].astype(int)
#data['Total_Registrations'] = data['Total_Registrations'].astype(int)
#data.info()

In [52]:
# 독립변수, 종속변수 분리

# 함수 사용하여 데이터 분리
independent = list(data.columns.difference(['Pet_Owners']))
# 'Pet_Owners'란 이름과 다른 열을 리스트로 뽑음

# 독립변수들
independent = data[independent]

# 독립변수 나누기
independent_registration = independent[['Total_Registrations']]
independent_reg_pop = independent[['Total_Registrations','Population']]
independent_reg_pop_h = independent[['Total_Registrations','Population', 'hospital']]

# 종속변수들
bound = data['Pet_Owners']

# 독립변수 확인
independent.columns

Index(['Population', 'Total_Registrations', 'h+b', 'h+p', 'h+p+b', 'hospital'], dtype='object')

In [53]:
# independent_registration

# 학습데이터(test)와 평가데이터(train) 8:2 비율로 만들기

x1 = independent_registration   # 독립변수 = 총등록수
x2 = independent_reg_pop        # 독립변수 = 총등록수, 인구수
x3 = independent_reg_pop_h      # 독립변수 = 총등록수, 인구수, 병원
y = bound                       # 종속변수 = 소유자수
train_x1, test_x1, train_x2, test_x2, train_x3, test_x3, train_y, test_y = train_test_split(x1, x2, x3, y, test_size =0.2, random_state=42 )

test_y =pd.DataFrame(test_y) 
print('train_x1.shape = ', train_x1.shape,'test_x1.shape = ', test_x1.shape) 
print('train_x2.shape = ', train_x2.shape,'test_x2.shape = ', test_x2.shape) 
print('train_x3.shape = ', train_x3.shape,'test_x3.shape = ', test_x3.shape) 
print('train_y.shape = ', train_y.shape, 'test_y.shape = ',test_y.shape) 

train_x1.shape =  (69, 1) test_x1.shape =  (18, 1)
train_x2.shape =  (69, 2) test_x2.shape =  (18, 2)
train_x3.shape =  (69, 3) test_x3.shape =  (18, 3)
train_y.shape =  (69,) test_y.shape =  (18, 1)


회귀분석 모델 구축
- x : registration

In [54]:
# 기본적인 random forest 모형

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score  # 정확도 함수

# clf : Classifier (분류)
clf = RandomForestClassifier(n_estimators=20, max_depth=5, random_state=20)
# n_estimators=20 20개의 의사결정 트리를 만든다 
#>> 이 값이 클수록 모델의 복잡도 증가, 성능 향상 증가
# max_depth=5 트리의 최대 깊이
#>> 모델의 깊이를 제한함으로써 모델의 과적합 방지
#>> 각 트리가 너무 깊게 성장하지 않도록 함

clf.fit(train_x1, train_y)
# fit 메서드를 호출하면 입력데이터와 출력데이터 사이의 관계를 학습

predict1 = clf.predict(test_x1) 
predict1 = pd.DataFrame(predict1)
#accuracy = accuracy_score(test_y, predict1)
#print(f'{accuracy:.2f}')
# predict1은 test_x에 대한 에측된 출력값을 계산
# accuracy_score 함수는 예측된 출력 값 predict1과 실제 정답 데이터 test_y를 
# 비교하여 분류 모델의 정확도를 계산

# 1에 가까울수록 모델이 더 정확하게 예측


In [59]:
# 실제 소유자 수와 추정된 소유자 수 비교


accuracy_list = []
for i in range(len(predict1)):
        print(i)
        print(f'실제 소유자 수 : {test_y.iloc[i,0]}')
        print(f'추정된 소유자 수: {predict1[0][i]}')
        print(f'예측의 정확도: {100*(1- abs(predict1[0][i] - test_y.iloc[i,0])/test_y.iloc[i,0]):.2f}%')
        accuracy = 100*(1- abs(predict1[0][i] - test_y.iloc[i,0])/test_y.iloc[i,0])
        accuracy_list.append(accuracy)
    
print('-'*30)
total_accuracy = sum(accuracy_list)/len(accuracy_list)

print(f'총 정확도: {total_accuracy:.2f}%')

0
실제 소유자 수 : 3807.0
추정된 소유자 수: 3871.0
예측의 정확도: 98.32%
1
실제 소유자 수 : 1344.0
추정된 소유자 수: 1919.0
예측의 정확도: 57.22%
2
실제 소유자 수 : 1534.0
추정된 소유자 수: 1621.0
예측의 정확도: 94.33%
3
실제 소유자 수 : 2511.0
추정된 소유자 수: 2511.0
예측의 정확도: 100.00%
4
실제 소유자 수 : 2268.0
추정된 소유자 수: 2488.0
예측의 정확도: 90.30%
5
실제 소유자 수 : 9596.0
추정된 소유자 수: 9567.0
예측의 정확도: 99.70%
6
실제 소유자 수 : 4399.0
추정된 소유자 수: 4551.0
예측의 정확도: 96.54%
7
실제 소유자 수 : 670.0
추정된 소유자 수: 670.0
예측의 정확도: 100.00%
8
실제 소유자 수 : 1873.0
추정된 소유자 수: 2039.0
예측의 정확도: 91.14%
9
실제 소유자 수 : 5156.0
추정된 소유자 수: 4551.0
예측의 정확도: 88.27%
10
실제 소유자 수 : 2318.0
추정된 소유자 수: 2039.0
예측의 정확도: 87.96%
11
실제 소유자 수 : 1019.0
추정된 소유자 수: 1127.0
예측의 정확도: 89.40%
12
실제 소유자 수 : 1222.0
추정된 소유자 수: 1361.0
예측의 정확도: 88.63%
13
실제 소유자 수 : 7221.0
추정된 소유자 수: 7477.0
예측의 정확도: 96.45%
14
실제 소유자 수 : 2612.0
추정된 소유자 수: 2361.0
예측의 정확도: 90.39%
15
실제 소유자 수 : 3929.0
추정된 소유자 수: 3871.0
예측의 정확도: 98.52%
16
실제 소유자 수 : 10512.0
추정된 소유자 수: 9567.0
예측의 정확도: 91.01%
17
실제 소유자 수 : 2256.0
추정된 소유자 수: 2361.0
예측의 정확도: 95.35%
-------------------

회귀분석 모델 구축
- x : registration + population

n_estimators=30, max_depth=10, random_state=100 >> 총 정확도: 92.04%

In [75]:
# clf : Classifier (분류)
clf = RandomForestClassifier(n_estimators=30, max_depth=10, random_state=100)


clf.fit(train_x2, train_y)
# fit 메서드를 호출하면 입력데이터와 출력데이터 사이의 관계를 학습

predict2 = clf.predict(test_x2) 
predict2 = pd.DataFrame(predict2)



accuracy_list = []
for i in range(len(predict2)):
        print(i)
        print(f'실제 소유자 수 : {test_y.iloc[i,0]}')
        print(f'추정된 소유자 수: {predict2[0][i]}')
        print(f'예측의 정확도: {100*(1- abs(predict2[0][i] - test_y.iloc[i,0])/test_y.iloc[i,0]):.2f}%')
        accuracy = 100*(1- abs(predict2[0][i] - test_y.iloc[i,0])/test_y.iloc[i,0])
        accuracy_list.append(accuracy)
    
print('-'*30)
total_accuracy = sum(accuracy_list)/len(accuracy_list)

print(f'총 정확도: {total_accuracy:.2f}%')

0
실제 소유자 수 : 3807.0
추정된 소유자 수: 3820.0
예측의 정확도: 99.66%
1
실제 소유자 수 : 1344.0
추정된 소유자 수: 1932.0
예측의 정확도: 56.25%
2
실제 소유자 수 : 1534.0
추정된 소유자 수: 1715.0
예측의 정확도: 88.20%
3
실제 소유자 수 : 2511.0
추정된 소유자 수: 2511.0
예측의 정확도: 100.00%
4
실제 소유자 수 : 2268.0
추정된 소유자 수: 2447.0
예측의 정확도: 92.11%
5
실제 소유자 수 : 9596.0
추정된 소유자 수: 9567.0
예측의 정확도: 99.70%
6
실제 소유자 수 : 4399.0
추정된 소유자 수: 4738.0
예측의 정확도: 92.29%
7
실제 소유자 수 : 670.0
추정된 소유자 수: 670.0
예측의 정확도: 100.00%
8
실제 소유자 수 : 1873.0
추정된 소유자 수: 1621.0
예측의 정확도: 86.55%
9
실제 소유자 수 : 5156.0
추정된 소유자 수: 4551.0
예측의 정확도: 88.27%
10
실제 소유자 수 : 2318.0
추정된 소유자 수: 2152.0
예측의 정확도: 92.84%
11
실제 소유자 수 : 1019.0
추정된 소유자 수: 917.0
예측의 정확도: 89.99%
12
실제 소유자 수 : 1222.0
추정된 소유자 수: 1247.0
예측의 정확도: 97.95%
13
실제 소유자 수 : 7221.0
추정된 소유자 수: 7477.0
예측의 정확도: 96.45%
14
실제 소유자 수 : 2612.0
추정된 소유자 수: 2361.0
예측의 정확도: 90.39%
15
실제 소유자 수 : 3929.0
추정된 소유자 수: 3820.0
예측의 정확도: 97.23%
16
실제 소유자 수 : 10512.0
추정된 소유자 수: 9567.0
예측의 정확도: 91.01%
17
실제 소유자 수 : 2256.0
추정된 소유자 수: 2304.0
예측의 정확도: 97.87%
--------------------

회귀분석 모델 구축
- x : registration + population + hospital

n_estimators=30, max_depth=10, random_state=100 >> 총 정확도: 86.86%

In [82]:
# clf : Classifier (분류)
clf = RandomForestClassifier(n_estimators=30, max_depth=10, random_state=100)


clf.fit(train_x3, train_y)
# fit 메서드를 호출하면 입력데이터와 출력데이터 사이의 관계를 학습

predict3 = clf.predict(test_x3) 
predict3 = pd.DataFrame(predict3)



accuracy_list = []
for i in range(len(predict3)):
        print(i)
        print(f'실제 소유자 수 : {test_y.iloc[i,0]}')
        print(f'추정된 소유자 수: {predict3[0][i]}')
        print(f'예측의 정확도: {100*(1- abs(predict3[0][i] - test_y.iloc[i,0])/test_y.iloc[i,0]):.2f}%')
        accuracy = 100*(1- abs(predict3[0][i] - test_y.iloc[i,0])/test_y.iloc[i,0])
        accuracy_list.append(accuracy)
    
print('-'*30)
total_accuracy = sum(accuracy_list)/len(accuracy_list)

print(f'총 정확도: {total_accuracy:.2f}%')

0
실제 소유자 수 : 3807.0
추정된 소유자 수: 3820.0
예측의 정확도: 99.66%
1
실제 소유자 수 : 1344.0
추정된 소유자 수: 1434.0
예측의 정확도: 93.30%
2
실제 소유자 수 : 1534.0
추정된 소유자 수: 2792.0
예측의 정확도: 17.99%
3
실제 소유자 수 : 2511.0
추정된 소유자 수: 2511.0
예측의 정확도: 100.00%
4
실제 소유자 수 : 2268.0
추정된 소유자 수: 2792.0
예측의 정확도: 76.90%
5
실제 소유자 수 : 9596.0
추정된 소유자 수: 9567.0
예측의 정확도: 99.70%
6
실제 소유자 수 : 4399.0
추정된 소유자 수: 4738.0
예측의 정확도: 92.29%
7
실제 소유자 수 : 670.0
추정된 소유자 수: 670.0
예측의 정확도: 100.00%
8
실제 소유자 수 : 1873.0
추정된 소유자 수: 2361.0
예측의 정확도: 73.95%
9
실제 소유자 수 : 5156.0
추정된 소유자 수: 4551.0
예측의 정확도: 88.27%
10
실제 소유자 수 : 2318.0
추정된 소유자 수: 2209.0
예측의 정확도: 95.30%
11
실제 소유자 수 : 1019.0
추정된 소유자 수: 917.0
예측의 정확도: 89.99%
12
실제 소유자 수 : 1222.0
추정된 소유자 수: 1247.0
예측의 정확도: 97.95%
13
실제 소유자 수 : 7221.0
추정된 소유자 수: 7477.0
예측의 정확도: 96.45%
14
실제 소유자 수 : 2612.0
추정된 소유자 수: 1717.0
예측의 정확도: 65.74%
15
실제 소유자 수 : 3929.0
추정된 소유자 수: 4074.0
예측의 정확도: 96.31%
16
실제 소유자 수 : 10512.0
추정된 소유자 수: 9567.0
예측의 정확도: 91.01%
17
실제 소유자 수 : 2256.0
추정된 소유자 수: 2511.0
예측의 정확도: 88.70%
--------------------